In [6]:
import os, sys, json, datetime, re  # Provides OS-dependent functionality, system-specific parameters, JSON handling, and date/time manipulation
import pandas as pd             # Provides data structures and data analysis tools
import numpy as np              # Supports large, multi-dimensional arrays and matrices
import requests
import time
from tqdm import tqdm
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
from bs4 import BeautifulSoup
from lxml import etree, html
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from cprl_functions.state_capture import thi_states,state_ref, state_coding, state_pat, state_abv_pat
from cprl_functions.text_printing import bordered
from cprl_functions.defined_functions import create_pk, extract_title_and_name, get_recent_file
from unidecode import unidecode

from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

## Set up

In [7]:
state_coding

{'AL': 10,
 'AK': 11,
 'AZ': 12,
 'AR': 13,
 'CA': 14,
 'CO': 15,
 'CT': 16,
 'DE': 17,
 'FL': 18,
 'GA': 19,
 'HI': 20,
 'ID': 21,
 'IL': 22,
 'IN': 23,
 'IA': 24,
 'KS': 25,
 'KY': 26,
 'LA': 27,
 'ME': 28,
 'MD': 29,
 'MA': 30,
 'MI': 31,
 'MN': 32,
 'MS': 33,
 'MO': 34,
 'MT': 35,
 'NE': 36,
 'NV': 37,
 'NH': 38,
 'NJ': 39,
 'NM': 40,
 'NY': 41,
 'NC': 42,
 'ND': 43,
 'OH': 44,
 'OK': 45,
 'OR': 46,
 'PA': 47,
 'RI': 48,
 'SC': 49,
 'SD': 50,
 'TN': 51,
 'TX': 52,
 'UT': 53,
 'VT': 54,
 'VA': 55,
 'WA': 56,
 'WV': 57,
 'WI': 58,
 'WY': 59,
 'DC': 60}

In [9]:
#driver set up
webdriver_path = r"C:\Users\clutz\hunt_env\chrome driver\chromedriver-win64\chromedriver.exe"
chrome_options = Options()
# chrome_options.add_argument('--headless')
# chrome_options.add_argument("--headless")  # Run in headless mode (no GUI)
# Set up WebDriver service
service = Service(webdriver_path)

#call on driver
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=chrome_options)


In [10]:
#list declaration
ok_coms_dfs = []

# Senate

In [11]:
senate_urls = """https://lis.virginia.gov/session-details/20251/committee-information/S04/committee-details
https://lis.virginia.gov/session-details/20251/committee-information/S05/committee-details
https://lis.virginia.gov/session-details/20251/committee-information/S09/committee-details"""

senate_urls = senate_urls.split('\n')
print(senate_urls)

['https://lis.virginia.gov/session-details/20251/committee-information/S04/committee-details', 'https://lis.virginia.gov/session-details/20251/committee-information/S05/committee-details', 'https://lis.virginia.gov/session-details/20251/committee-information/S09/committee-details']


In [12]:
com_dfs = []
for url in senate_urls:
    driver.get(url)
    time.sleep(3)
    #general info
    comname = driver.find_element(By.CSS_SELECTOR, 'div.details-header')
    comname = comname.text.replace('Senate', '').strip()
    
    #
    members = driver.find_element(By.XPATH, "//*[@class='details-section']")
    members = members.find_elements(By.TAG_NAME, "a")
    com_mems = []
    for m in members:
        
        
        if re.search(r'/(.+/)', m.text):
            position = re.extract(r'\((.+)\)')
            position = position.group(1)
        else:
            position = 'member'
        
        url = m.get_attribute('href')

        df = pd.DataFrame(({'url':[url],'position':[position]}))
        com_mems.append(df)
    com_df = pd.concat(com_mems).reset_index(drop=True)
    com_df['com_name'] = comname
    com_dfs.append(com_df)


va_senate_coms = pd.concat(com_dfs)

    
    # print(len(members))
    # for m in members:
    # members = members.find_elements(By.CSS_SELECTOR, 'a')

    # for m in members:

In [16]:
def create_pk2(state, chamber, district, mls):
    
    if not re.search(r'^[A-Z]{2}$', state):
        
        error = 'func error: state is wrong'
        return print(error)
        

    state_code = state_coding.get(state)
    
    if not re.search(r'[Hh]ouse|[Ss]enate', chamber):
        error = 'func error: chamber wrong'
        return print(error)
    elif re.search(r'[Hh]ouse',chamber):
        chamber_code = 0
    else:
        chamber_code = 1

    if not re.search(r'^\d+$', district):
        return print('func error: district is more than just numbers')
    else:
        dist_code = str(district).strip().zfill(3)

    if mls == 'y':
        print('im not set up for that yet')
    else:
        seat = '00'

    fpk = int(f'{state_code}{chamber_code}{dist_code}{seat}')
    return fpk

    

In [17]:
!jupyter nbextension enable --py widgetsnbextension


Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


In [18]:
from tqdm.notebook import tqdm_notebook, tqdm
tqdm.pandas()
import pandas as pd 


# tqdm_notebook().pandas()

# df = pd.DataFrame({'a' : ['foo', 'bar'], 'b' : ['spam', 'eggs']}) 
# df.progress_apply(lambda row: row['a'] + row['b'], axis = 1)
# .progress_apply(lambda row: row['a'] + row['b'], axis = 1)

lens = []
va_senate_coms['name'] = np.nan
va_senate_coms['district'] = np.nan
va_senate_coms['fpk'] = np.nan 

# for row in va_senate_coms.itertuples(index=True):
for row in tqdm_notebook(va_senate_coms.itertuples(index=True),total=len(va_senate_coms)):
    driver.get(row.url)
    # time.sleep(3)
    wait = WebDriverWait(driver, 10)  # Waits up to 10 seconds
    # print(row)

    element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, ".details-header")))
    # print(element.text)
    ele_split = element.text.split('-')
    
    #name the split strings
    name_string = ele_split[0]
    dis_string = ele_split[1]

    name = name_string.split('(')[0]
    district = re.search(r'District (\d+)', dis_string).group(1)

    fpk = create_pk2("VA", "Senate", district, 'n')
    print(fpk)
    va_senate_coms.loc[row.Index,'name'] = name
    va_senate_coms.loc[row.Index,'district'] = district
    va_senate_coms.loc[row.Index,'fpk'] = fpk


    # re.split()
# print(lens)


  0%|          | 0/45 [00:00<?, ?it/s]

55101500
55101800
55102300
55100400
55100800
55100600
55104000
55103800
55101400
55100300
55101300
55101600
55102700
55103600
55101900
55101800
55101100
55102300
55103500
55103900
55102600
55100200
55102500
55102800
55104000
55103400
55102900
55103800
55100600
55101500
55104000
55102600
55102800
55103400
55102000
55102200
55100300
55101700
55101600
55102100
55103300
55103100
55100100
55103700
55101900


## Cleaning

In [26]:
pop_fpk = va_senate_coms.pop('fpk')


In [ ]:
va_senate_coms.insert(0,'fpk',pop_fpk)


In [31]:

va_senate_coms = va_senate_coms.drop('url', axis = 1)

# House Data Gathering
this was manually pulled together, just pulling in here to merge with scraped data

In [45]:
file = r'c:\Users\clutz\THE HUNT INSTITUTE\The Hunt Institute Team Site - Documents\Development (formerly Grants Management)\!Administrative\Christian\THII\legislator data\committee info\committee update files\VA_house_comms.xlsx'

va_house_data = pd.read_excel(file)
va_house_data = va_house_data.rename(columns = {'com name': 'com_name', 'role': 'position'})
va_house_data

,chamber,com_name,name,position,district
0,House,Appropriations - Higher Education Subcommittee,Betsy B. Carr,Chair,78
1,House,Appropriations - Higher Education Subcommittee,Mark D. Sickles,Member,17
2,House,Appropriations - Higher Education Subcommittee,David A. Reid,Member,28
3,House,Appropriations - Higher Education Subcommittee,Rodney T. Willett,Member,58
4,House,Appropriations - Higher Education Subcommittee,Alex Q. Askew,Member,95
...,...,...,...,...,...
66,House,Health and Human Services,Howard Otto Wachsmann Jr.,Member,83
67,House,Health and Human Services,Kimberly A. Taylor,Member,82
68,House,Health and Human Services,N. Baxter Ennis,Member,89
69,House,Health and Human Services,Geary M. Higgins,Member,30


In [46]:
print(va_senate_coms.columns)

Index(['fpk', 'position', 'com_name', 'name', 'district'], dtype='object')


## cleaning
- adding pk

In [54]:

# for row in va_house_data.itertuples(index=True):
va_house_data['fpk'] = np.nan
for row in tqdm_notebook(va_house_data.itertuples(index=True),total=len(va_house_data)):
# for row in tqdm_notebook(va_house_data.items()):
    # print(row.name)
    # print(row.district)
    
    


    fpk = create_pk2("VA", "House", str(row.district), 'n')
    # print(fpk)
    va_house_data.loc[row.Index,'fpk'] = fpk

pop_fpk = va_house_data.pop('fpk')
va_house_data.insert(0,'fpk',pop_fpk)


  0%|          | 0/71 [00:00<?, ?it/s]

In [55]:
va_house_data

,fpk,chamber,com_name,name,position,district
0,55007800.0,House,Appropriations - Higher Education Subcommittee,Betsy B. Carr,Chair,78
1,55001700.0,House,Appropriations - Higher Education Subcommittee,Mark D. Sickles,Member,17
2,55002800.0,House,Appropriations - Higher Education Subcommittee,David A. Reid,Member,28
3,55005800.0,House,Appropriations - Higher Education Subcommittee,Rodney T. Willett,Member,58
4,55009500.0,House,Appropriations - Higher Education Subcommittee,Alex Q. Askew,Member,95
...,...,...,...,...,...,...
66,55008300.0,House,Health and Human Services,Howard Otto Wachsmann Jr.,Member,83
67,55008200.0,House,Health and Human Services,Kimberly A. Taylor,Member,82
68,55008900.0,House,Health and Human Services,N. Baxter Ennis,Member,89
69,55003000.0,House,Health and Human Services,Geary M. Higgins,Member,30


In [ ]:
# QA/QC

In [63]:
# clean and reorganize columns
dfs = [va_house_data,va_senate_coms]
for i,df in enumerate(dfs):
    # print(type(df))
    df = df.loc[:,['fpk', 'com_name', 'name','position','district']]
    # print(type(df))
    dfs[i] = df
    print(df.columns)
va_com_info = pd.concat(dfs).reset_index(drop = True)



Index(['fpk', 'com_name', 'name', 'position', 'district'], dtype='object')
Index(['fpk', 'com_name', 'name', 'position', 'district'], dtype='object')


In [ ]:
#set up
ref_path = r"C:\Users\clutz\THE HUNT INSTITUTE\The Hunt Institute Team Site - Documents\Development (formerly Grants Management)\!Administrative\Christian\THII\legislator data\key_creation\2025"
leg_ref =pd.read_excel(get_recent_file("*", ref_path))
leg_ref = leg_ref[leg_ref['state abbreviation'] == "VA"]
# print(leg_ref.columns)
# print(leg_ref.head())
# leg_ref


In [67]:
def get_subset(df, fpk_col, fpk):
    df_new = df[df[fpk_col] == fpk]
    return df_new

In [74]:
#checking that all are assigned correctly
lens = []
for row in va_com_info.itertuples(index=False):
    sub_df = get_subset(leg_ref, 'full_pk', int(row.fpk))
    lens.append(len(sub_df))
    if len(sub_df) >1:
        print(row)
        print(f'sub df :')
        for row in sub_df.itertuples(index=False):
            print(row)
    elif len(sub_df) <1:
        print(row)
        print(sub_df)
        
    # print(sub_df)
print(list(set(lens)))


"""
if this returns anything other than [1] that means that the fpk match isnt fully clean
"""

[1]


'\nif this returns anything other than [1] that means that the fpk match isnt fully clean\n'

In [77]:
#export
com_folder = r'C:\Users\clutz\THE HUNT INSTITUTE\The Hunt Institute Team Site - Documents\Development (formerly Grants Management)\!Administrative\Christian\THII\legislator data\committee info\committee update files\full_pk_assigned'
file_name = os.path.join(com_folder,"va_com_info.xlsx")
va_com_info.to_excel(file_name, index=False)